# 目的
PILモジュールを利用したドット絵の作成テストを行う。   

In [ ]:
from PIL import Image
from PIL import ImageDraw
import numpy as np
import copy
import pandas as pd

In [ ]:
for line in Image.new.__doc__.split("\n"):
    print(line)

In [ ]:
def make_image(screen, bgcolor, filename, drawfunc, issave=False, **kwargs):
    """
    画像ファイルを作成
    """
    img = Image.new('RGB', screen, bgcolor)

    # 間隔（5～32くらい）
#     gap = 8

    img = drawfunc(img, **kwargs)
    plt.imshow(img, origin="normal")
    is_axis = kwargs.get("axis", True)
    if not is_axis:
        plt.axis("off")
    if issave:
        img.save(filename)
    return img

## 参考：<http://qiita.com/suto3/items/87af35517f2a8c3bc22e>

In [ ]:
def drawing(img, gap):
    """
    描画（ファイルに書く画像データを編集する）
    点を打つ
    """
    x,y = img.size
    draw = ImageDraw.Draw(img)
    for i in range(0,x,gap):
        for j in range(0,y,gap):
            # ↓こんなパターンを描く
            #■■■□
            #■□■□
            #■■■□
            #□□□□
            draw.point((i  , j  ),(0xff,0x00,0x00)) #red
            draw.point((i+1, j  ),(0xff,0x00,0x00)) #red
            draw.point((i+2, j  ),(0xff,0x00,0x00)) #red
            draw.point((i+3, j  ),(0x00,0xff,0x00)) #green

            draw.point((i  , j+1),(0xff,0x00,0x00)) #red
            draw.point((i+1, j+1),(0xff,0xff,0x00)) #yellow
            draw.point((i+2, j+1),(0xff,0x00,0x00)) #red
            draw.point((i+3, j+1),(0x00,0xff,0x00)) #green

            draw.point((i  , j+2),(0xff,0x00,0x00)) #red
            draw.point((i+1, j+2),(0xff,0x00,0x00)) #red
            draw.point((i+2, j+2),(0xff,0x00,0x00)) #red
            draw.point((i+3, j+2),(0x00,0xff,0x00)) #green

            draw.point((i  , j+3),(0x00,0xff,0x00)) #green
            draw.point((i+1, j+3),(0x00,0xff,0x00)) #green
            draw.point((i+2, j+3),(0x00,0xff,0x00)) #green
            draw.point((i+3, j+3),(0x00,0xff,0x00)) #green

    return img

In [ ]:
if __name__ == '__main__':
    # 画像のサイズ
    screen = (800,600)

    # 画像の背景色（RGB）
    bgcolor=(0xdd,0xdd,0xdd)

    # 保存するファイル名（ファイル形式は、拡張子から自動的に判別する）
    filename = "../images/image-set-flower.png"

    make_image(screen, bgcolor, drawing, filename, gap=8)

## 棒を描画してみる

In [ ]:
def drawBar(img, topleft=[1,1], width=10, height=20, **kwargs):
    y, x = img.size
    if x < width or y < height:
        raise ValueError
    draw = ImageDraw.Draw(img)
    start = tuple(topleft)
    end = tuple([topleft[0], topleft[1]+height])
    draw.line((start, end),(0x00,0x00,0x00),  width)
    return img

In [ ]:
if __name__ == '__main__':
    # 画像のサイズ
    screen = (50,50)

    # 画像の背景色（RGB）
    bgcolor=(0xff,0xff,0xFf)

    # 保存するファイル名（ファイル形式は、拡張子から自動的に判別する）
    filename = "../images/image-bar.png"

    make_image(screen, bgcolor, drawBar, filename, topleft=[20, 10])

## ひげをつけてみる
箱ひげ図を描画する

In [ ]:
def drawBarWithJoes(img, topleft=[0,0], width=10, height=20, barcolor=(0x00,0x00,0x00), **kwargs):
    y, x = img.size
    if x < width or y < height:
        raise ValueError
    draw = ImageDraw.Draw(img)
    
    ### Bar
    start_bar = tuple([topleft[0]+width//2, topleft[1]])
    end_bar = tuple([start_bar[0], topleft[1]+height])
    draw.line((start_bar, end_bar), barcolor, width)
    
    ### Joes
    joeDistanceTop = kwargs.get("joe_distance_top", 10)
    joeDistanceBottom = kwargs.get("joe_distance_bottom", joeDistanceTop)
    joeLength = kwargs.get("joe_length", width)
    joeWidth = kwargs.get("joe_width", 1)
    joeColor = kwargs.get("joe_color", barcolor)
    
    # Top joe
    start = tuple([topleft[0]+(width-joeLength+1)//2, topleft[1]-joeDistanceTop])
    end = tuple([start[0]+joeLength-1, start[1]])
    draw.line((start, end), joeColor, joeWidth)
    
    # Connection to bar
    start = tuple([start_bar[0], start_bar[1]-joeDistanceTop])
    draw.line((start, start_bar), joeColor, joeWidth)
    
    # Bottom joe
    start = tuple([topleft[0]+(width-joeLength+1)//2, end_bar[1]+joeDistanceBottom])
    end = tuple([start[0]+joeLength-1, start[1]])
    draw.line((start, end), joeColor, joeWidth)
    
    # Connection to bar
    end = tuple([end_bar[0], end_bar[1]+joeDistanceBottom])
    draw.line((end, end_bar), joeColor, joeWidth)
    
    return img

In [ ]:
if __name__ == '__main__':
    # 画像のサイズ
    screen = (60,60)

    # 画像の背景色（RGB）
    bgcolor=(0xff,0xff,0xFf)

    # 保存するファイル名（ファイル形式は、拡張子から自動的に判別する）
    filename = "../images/image-bar.png"

    make_image(screen, bgcolor, filename, drawBarWithJoes, topleft=[20, 20], width=11)

## OHLC図

In [ ]:
def drawOhlcBox(img, ohlc, start=[0,0], width=10, barcolor=(0x00,0x00,0x00), return_next=False, **kwargs):
    """
    ohlc: (1,4) array
    start: start point (open)
    width
    barcolor
    kwargs
    """
    draw = ImageDraw.Draw(img)
    
    ### Define each start point
    ## Bar
    # start_bar[1] = start[1] + (open - open), end_bar[1] = start[1] + (close - open)
    start_bar = tuple([start[0]+width//2, start[1]])
    
    end_bar   = tuple([start_bar[0], start[1] + ohlc[3] - ohlc[0]])
    
    ## Line between high and low
    # if open > close then start_hl[1] = start_bar[1] + (high - open), end_hl[1] = end_bar[1] + (low - close)
    # if open < close then start_hl[1] = start_bar[1] + (low - open), end_hl[1] = end_bar[1] + (high - close)
    if ohlc[0] >= ohlc[3]:
        start_hl = tuple([start_bar[0], start_bar[1] + (ohlc[1] - ohlc[0])])
        end_hl   = tuple([start_bar[0], end_bar[1] + (ohlc[2] - ohlc[3])])
    else:
        start_hl = tuple([start_bar[0], start_bar[1] + (ohlc[2] - ohlc[0])])
        end_hl   = tuple([start_bar[0], end_bar[1] + (ohlc[1] - ohlc[3])])
    
    ### Draw
    draw.line((start_bar, end_bar), barcolor, width)
    draw.line((start_hl, end_hl), barcolor, 1)
    
#     ### Joes
#     joeDistanceTop = kwargs.get("joe_distance_top", 10)
#     joeDistanceBottom = kwargs.get("joe_distance_bottom", joeDistanceTop)
#     joeLength = kwargs.get("joe_length", width)
#     joeWidth = kwargs.get("joe_width", 1)
#     joeColor = kwargs.get("joe_color", barcolor)
    
#     # Top joe
#     start = tuple([topleft[0]+(width-joeLength+1)//2, topleft[1]-joeDistanceTop])
#     end = tuple([start[0]+joeLength-1, start[1]])
#     draw.line((start, end), joeColor, joeWidth)
    
#     # Connection to bar
#     start = tuple([start_bar[0], start_bar[1]-joeDistanceTop])
#     draw.line((start, start_bar), joeColor, joeWidth)
    
#     # Bottom joe
#     start = tuple([topleft[0]+(width-joeLength+1)//2, end_bar[1]+joeDistanceBottom])
#     end = tuple([start[0]+joeLength-1, start[1]])
#     draw.line((start, end), joeColor, joeWidth)
    
#     # Connection to bar
#     end = tuple([end_bar[0], end_bar[1]+joeDistanceBottom])
#     draw.line((end, end_bar), joeColor, joeWidth)
    
    if not return_next:
        return img
    else:
        start_next = [start[0] + width, end_bar[1]]
        return img, start_next

def drawOhlcBoxes(img, ohlc=None, start=[0, 0], width=11, barcolors=(0x00,0x00,0x00), **kwargs):
    """
    topleft_startは上ひげのtopleftになる
    """
    if ohlc is None:
        raise ValueError("ohlc must be inputted.")
    elif not isinstance(ohlc, np.ndarray):
        raise TypeError("ohlc must be a (N, 4) array.")
    if not isinstance(barcolors, list):
        bar_colors = [barcolors] * len(ohlc)
    else:
        bar_colors = barcolors
    
    k_wargs = copy.deepcopy(kwargs)
    if k_wargs.get("width") is not None:
        del k_wargs["width"]
    if k_wargs.get("height") is not None:
        del k_wargs["height"]
    if k_wargs.get("barcolors") is not None:
        del k_wargs["barcolors"]
        
    start_next = start
    for ii in range(len(ohlc)):
        img, start_next = drawOhlcBox(img, ohlc[ii], start_next, width, bar_colors[ii], True, **k_wargs)
        
    return img

### 一個だけ

In [ ]:
if __name__ == '__main__':
    # 画像のサイズ
    screen = (150,150)

    # 画像の背景色（RGB）
    bgcolor=(0xff,0xff,0xFf)

    # 保存するファイル名（ファイル形式は、拡張子から自動的に判別する）
    filename = "../images/image-bar.png"
    
    ohlc = np.array([[120, 130, 80, 100]])
    make_image(screen, bgcolor, filename, drawOhlcBox, ohlc=ohlc[0], start=[60, 60], width=11)

### 複数

In [ ]:
if __name__ == '__main__':
    # 画像のサイズ
    screen = (150,150)

    # 画像の背景色（RGB）
    bgcolor=(0xff,0xff,0xFf)

    # 保存するファイル名（ファイル形式は、拡張子から自動的に判別する）
    filename = "../images/image-bar.png"
    
    ohlc = np.array([[120, 130, 80, 100], [100, 120, 90, 110], [110, 150, 110, 130]])
    make_image(screen, bgcolor, filename, drawOhlcBoxes, ohlc=ohlc, start=[20, 60], width=11)

## 実際のデータを用いたOHLC図
実際のデータは浮動小数であり、小数点第３位くらいまでデータが存在する。   
OHLC図を描くキャンパスのサイズが固定であるとして、その中に一連のOHLCデータを描画するために、次の方針を立てる。   
まず与えられたOHLCを規格化する。OHLCのmaxとminを取ると、
$$
M, m = max(OHLC), min(OHLC)
$$
これらを用いて、OHLCの値が0-(縦方向のpixel数)になるように規格化する。
$$
\bar{OHLC} = int((OHLC - m) \times pixel / (M-m))
$$
これでOHLC図の縦方向のサイズが決まる。横方向は(横方向のpixel数)/(描画するOHLC図の数)でOHLC図の幅を決めればよい。   
スタートは(0, $\bar{OHLC}[0,0]$)である。   

### ほかのものを描画する場合
VolumeやSMAなどを描画する場合は、次のようにするか。

* Volume: Volumeのmax/minをとってOHLCのように規格化。描画はlineで行う
* SMA: OHLCとSMAをまとめたもののmax/minをとって、OHLCとSMAを規格化。

In [ ]:
def drawOhlcBoxes2(img, ohlc=None, start=[0, 0], width=11, barcolors=(0x00,0x00,0x00), **kwargs):
    """
    topleft_startは上ひげのtopleftになる
    """
    if ohlc is None:
        raise ValueError("ohlc must be inputted.")
    elif not isinstance(ohlc, np.ndarray):
        raise TypeError("ohlc must be a (N, 4) array.")
    if not isinstance(barcolors, list):
        bar_colors = [barcolors] * len(ohlc)
    else:
        bar_colors = barcolors
    
    k_wargs = copy.deepcopy(kwargs)
    if k_wargs.get("width") is not None:
        del k_wargs["width"]
    if k_wargs.get("height") is not None:
        del k_wargs["height"]
    if k_wargs.get("barcolors") is not None:
        del k_wargs["barcolors"]
    
    start_next = start
    for ii in range(len(ohlc)):
        start_ohlc = [start_next[0], ohlc[ii, 0]]
        img, start_next = drawOhlcBox(img, ohlc[ii], start_ohlc, width, bar_colors[ii], True, **k_wargs)
        
    return img

In [ ]:
data = pd.read_csv("../data/FXTF/USDJPY-cd1_20170806_k030.csv")
data.head(10)

In [ ]:
ohlc = data[["open", "high", "low", "close"]].as_matrix()

h, w = 100, 100
ii = 0
nbr_of_ohlc = 10
buff = ohlc[ii:ii+nbr_of_ohlc]
ohlc_bar = np.array((buff - buff.min()) * h / (buff.max() - buff.min()), dtype=int)

In [ ]:
if __name__ == '__main__':
    # 画像のサイズ
    screen = (h, w)

    # 画像の背景色（RGB）
    bgcolor=(0x00, 0x00, 0x00)
    barcolors = (0xff, 0xff, 0xff)

    # 保存するファイル名（ファイル形式は、拡張子から自動的に判別する）
    filename = "../images/image-bar.png"
    img = make_image(screen, bgcolor, filename, drawOhlcBoxes2, ohlc=ohlc_bar, start=[0, ohlc_bar[0,0]], width=w//nbr_of_ohlc, barcolors=barcolors, axis=False, issave=True)

In [ ]:
img.save()